In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if openai_api_key:
    print("Openai api key loaded successfully")
else:
    print("Failed to load openai api key")
if gemini_api_key:
    print("Gemini api key loaded successfully")
else:
    print("Failed to load gemini api key")
if groq_api_key:
    print("Groq api key loaded successfully")
else:
    print("Failed to load groq api key")

Openai api key loaded successfully
Gemini api key loaded successfully
Groq api key loaded successfully


In [4]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"

openai = OpenAI()
gemini = OpenAI(base_url=gemini_url, api_key=gemini_api_key)
groq = OpenAI(base_url=groq_url, api_key=groq_api_key)

In [5]:
openai_system_prompt = "You are a helpful assistant"

def message_openai(prompt):
    messages = [
        {"role": "system", "content": openai_system_prompt},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [6]:
message_openai("What is today's date?")

"Today's date is June 12, 2024."

In [7]:
def convert_to_uppercase(text):
    print(f"Function convert_to_uppercase is called with {text}")
    return text.upper()

In [8]:
convert_to_uppercase("hello")

Function convert_to_uppercase is called with hello


'HELLO'

In [9]:
gr.Interface(fn=convert_to_uppercase, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [10]:
input_message = gr.Textbox(label="Your message:", info="Enter your message to convert in uppercase", lines=7)
output_message = gr.Textbox(label="Output:", lines=8)

gr.Interface(
    fn=convert_to_uppercase,
    title="Convert to uppercase",
    inputs=[input_message],
    outputs=[output_message],
    examples=["hello", "make me uppercase"],
    flagging_mode="never"
    ).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [11]:
input_message = gr.Textbox(label="Your message:", info="Enter a message for gpt-4.1-mini", lines=7)
output_message = gr.Textbox(label="Output:", lines=8)

gr.Interface(
    fn=message_openai,
    title="Openai",
    inputs=[input_message],
    outputs=[output_message],
    examples=["hello", "Hi! there"],
    flagging_mode="never"
    ).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [12]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

input_message = gr.Textbox(label="Your message:", info="Enter a message for gpt-4.1-mini", lines=7)
output_message = gr.Markdown(label="Response:")

gr.Interface(
    fn=message_openai,
    title="Your Assistant",
    inputs=[input_message],
    outputs=[output_message],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
    ],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [13]:
def stream_openai(prompt):
    messages = [
        {"role": "system", "content": openai_system_prompt},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [14]:
input_message = gr.Textbox(label="Your message:", info="Enter a message for gpt-4.1-mini", lines=7)
output_message = gr.Markdown(label="Response:")

gr.Interface(
    fn=stream_openai,
    title="Your Assistant",
    inputs=[input_message],
    outputs=[output_message],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
    ],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [15]:
def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = gemini.chat.completions.create(
        model="gemini-2.5-flash",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content
        yield result

In [16]:
input_message = gr.Textbox(label="Your message:", info="Enter a message for gemini-2.5-flash", lines=7)
output_message = gr.Markdown(label="Response:")

gr.Interface(
    fn=stream_gemini,
    title="Your Assistant",
    inputs=[input_message],
    outputs=[output_message],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
    ],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [20]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_openai(prompt=prompt)
    elif model == "Gemini":
        result = stream_gemini(prompt=prompt)
    else:
        raise ValueError("unknown model")
    yield from result 

In [21]:
input_message = gr.Textbox(label="Your message:", info="Enter a message for selected AI model", lines=7)
model_selector = gr.Dropdown(["GPT", "Gemini"], label="Select a model:", value="GPT")
output_message = gr.Markdown(label="Response:")

gr.Interface(
    fn=stream_model,
    title="Your Assistant of selected model",
    inputs=[input_message, model_selector],
    outputs=[output_message],
    examples=[
        ["Explain the Transformer architecture to a layperson", "GPT"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "Gemini"],
    ],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [22]:
from scraper import fetch_website_contents

In [23]:
system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [24]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url=url)
    if model == "GPT":
        result = stream_openai(prompt=prompt)
    elif model == "Gemini":
        result = stream_gemini(prompt=prompt)
    else:
        raise ValueError("Unknown model")
    
    yield from result

In [25]:
input_company_name = gr.Textbox(label="Company Name:")
input_url = gr.Textbox(label="Enter company url including http:// or https://")
input_model_selector = gr.Dropdown(["GPT", "Gemini"], label="Select model:")
output_message = gr.Markdown(label="Brochure:")

gr.Interface(
    fn=stream_brochure,
    title="Company Brochure Generator",
    inputs=[input_company_name, input_url, input_model_selector],
    outputs=[output_message],
    examples=[
        ["Hugging Face", "https://huggingface.co", "GPT"],
        ["Edward Donner", "https://edwarddonner.com", "Gemini"]
    ],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
